## Callable objects
A callable in Python is any object that you can call using a pair of parentheses and a series of arguments if required. You’ll find different examples of callables in your daily interaction with Python. Some of them include:
* Built-in functions and classes
* User-defined functions that you create with the def keyword
* Anonymous functions that you write using the lambda keyword
* The constructors of your custom classes
* Instance, class, and static methods
* Instances of classes that implement the .__call__() method
* Closures that you return from your functions
* Generator functions that you define using the yield keyword
* Asynchronous functions and methods that you create with the async keyword

I think only the objects we create ourselves using some class are not callable by default in python

In [2]:
# We can check if the instance has the __call__ method implemented in it
# using the dir function

# internal function
print("INTERNAL =", dir(len))

print("\n******************************")

# user defined functions
def hi():
    print("hi")
print("USER DEFINED ==",dir(hi))

print("\n*****************")

# T%his can also be called using that __call__ method
hi.__call__()

INTERNAL = ['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__text_signature__']

******************************
USER DEFINED == ['__annotations__', '__builtins__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__getstate__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']

*****************
hi


#### Class example
The class constructor of SampleClass falls back to using ***type.\__call\__()***. That’s why you can call SampleClass() to get a new instance, class constructors are callable objects that return new instances of the underlying class.

In [4]:
class Simple:
    def method(self):
        return "hello"
    
# Now lets see what is callable

# this will check if the constructor function of class is cdallable
# The class constructor of SampleClass falls back to using type.__call__(). That’s why you can call SampleClass() to get a new instance. 
# So, class constructors are callable objects that return new instances of the underlying class.
print("constructor functn ==",callable(Simple))

simpleObj = Simple()

# the method in class
print("method functn ==",callable(simpleObj.method))

# if the instance is callable
print("instance ==",callable(simpleObj))


constructor functn == True
method functn == True
instance == False


#### Inference
the instances are not callable by default

## Creating a class that returns callable objects
The class below provides the instances that have data (**instance attribute**) count instantiated to 0 and whenever the instance is called then the counter increases

In [7]:
class Counter:
    def __init__(self):
        self.count = 0

    def increment(self):
        self.count += 1

    def __call__(self):
        self.increment()

In [8]:
count1 = Counter()
print(count1.count)

count1()
print(count1.count)

count1()
print(count1.count)

count1()
print(count1.count)

# This also increments the same property
count1.increment()
print(count1.count)

0
1
2
3
4


#### More interesting example

There is no limitation we can get attribute to __call__ and also return values from them

In [11]:
class PowerFactory:
    def __init__(self, exponent=2):
        self.exponent = exponent
    def __call__(self, base):
        return base**self.exponent

In [12]:
power2 = PowerFactory(2)
power2(8)

64

In [13]:
power3 = PowerFactory(3)
power3(8)

512

## Understanding the Difference: .\__init\__() vs .\__call\__()
* Note that \__init\__ is called as the constructor function of the class
* and the \__call\__ is called when the instance of the function is called

In [15]:
class Demo:
    def __init__(self, attr):
        print(f"Initialize an instance of {self.__class__.__name__}")
        self.attr = attr
        print(f"{self.attr = }")

    def __call__(self, arg):
        print(f"Call an instance of {self.__class__.__name__} with {arg}")


In [16]:
demoEx = Demo("passed attribute to constructor")

Initialize an instance of Demo
self.attr = 'passed attribute to constructor'


In [17]:
demoEx("attribute: passed to the instance of the class demo")

Call an instance of Demo with attribute: passed to the instance of the class demo


You’ll find .\__init\__() in all Python classes. Some classes will have an explicit implementation, and others will inherit the method from a parent class. In many cases, object is the class that provides this method:

Remember that object is the ultimate parent class of all Python classes. So, even if you don’t define an explicit .__init__() method in one of your custom classes, that class will still inherit the default implementation from object.

## Putting Python’s .\__call\__() Into Action

### Writing Stateful Callables
means the functions that have states attached to them eg memoization, logger
To solve this problem we use **closures** to get a enclosed state with any function and then use it taking advantage of the state attached as the closure to it


**NOTE:** Both the soln saves us from using some global variable 

#### Using the closures

In [22]:
def cumAvg():
    data = []
    def avg(num):
        data.append(num)
        return sum(data)/len(data)
    return avg

In [23]:
# by calling cumAvg we get a function with the state in closure that can is used to calc cumulative avg
# Closures can be used to avoid global values and provide data hiding, 
# and can be an elegant solution for simple cases with one or few methods.

streamAvg = cumAvg()

In [24]:
# Now we can use this to get the avg of the all passed values and keeps the record of them as well
avg1 = streamAvg(2)
print(avg1)
print(streamAvg.__closure__)

2.0
(<cell at 0x7f7e18669ba0: list object at 0x7f7dca7a3580>,)


In [25]:
avg1 = streamAvg(3)
print(avg1)
print(streamAvg.__closure__)

2.5
(<cell at 0x7f7e18669ba0: list object at 0x7f7dca7a3580>,)


#### Now using the callable instances

* Another interesting advantage over closures is that now you have direct access to the current data through the .data attribute
* makes your code easier to reason about

In [27]:
class CumAvg:
    def __init__(self):
        self.data = []
    def __call__(self, number):
        self.data.append(number)
        return sum(self.data)/len(self.data)

stream_avg = CumAvg()

print(stream_avg(10))
print(stream_avg(20))
print(stream_avg(20))
print(stream_avg(10))        

10.0
15.0
16.666666666666668
15.0


### Caching Computed Values
Another common use case of callable instances is when you need a stateful callable that caches computed data between calls. This will be handy when you need to optimize some algorithms.

For example, say that you want to compute the factorial of a given number. Because you plan to run this computation multiple times, you need to make it efficient. A way to do this is to cache the already-computed values so that you don’t have to recompute them all the time.

In [29]:
class Factorial:
    def __init__(self):
        self.cache = {0:1, 1:1}
    def __call__(self, number):
        if number not in self.cache:
            self.cache[number] = number * self(number - 1)
            return self.cache[number]
        else:
            return self.cache[number]

In [30]:
%%time

factorialCalculator = Factorial()
factorialCalculator(50)

CPU times: user 50 µs, sys: 6 µs, total: 56 µs
Wall time: 63.4 µs


30414093201713378043612608166064768844377641568960512000000000000

In [31]:
# a recursive function

def fib(num):
    if(num<=1 ):
        return 1
    else:
        return num * fib(num-1)

In [32]:
%%time
fib(50)

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 11.9 µs


30414093201713378043612608166064768844377641568960512000000000000

### When Class has a single behaviour
Another use case where .]__call\__() can help you improve your APIs is when you have a class whose primary purpose is to provide a single action or behavior. For example, say you want a Logger class that takes care of logging messages to a file:

In [34]:
class Logger:
    def __init__(self, filename):
        self.filename = filename

    def __call__(self, message):
        with open(self.filename, mode="a", encoding="utf-8") as log_file:
            log_file.write(message + "\n")

In [35]:
logger = Logger("log_test.txt")
logger("This is a cool callable instance for logger")

## Exploring Advanced Use Cases of .__call\__()

### Writing Class-Based Decorators
Python’s **decorators are callables** that **take another callable** as an argument and extend its behavior without explicitly modifying its code. Decorators provide an excellent tool for adding new functionality **to existing callables**.

It’s pretty common to find and write function-based decorators. However, you can also write class-based decorators by taking advantage of the .\__call\__() special method.

**Classes are already callable so they can be decorated with any decorator** 



In [39]:
import time

class ExecutionTimer:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        start = time.perf_counter()
        result = self.func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{self.func.__name__}() took {(end - start) * 1000:.4f} ms")
        return result
    
# this class can be used as the decorator that takes a function (in decorator __init__) and then using \__call\__ ads the additional functionalities
# and returns the function

In [66]:
@ExecutionTimer
def square_numbers(numbers):
    return [number ** 2 for number in numbers]


print(square_numbers(list(range(100))))

square_numbers() took 0.0115 ms
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


In [68]:
import time

class ExecutionTimer:
    def __init__(self, repetitions=1):
        self.repetitions = repetitions

    def __call__(self, func):
        def timer(*args, **kwargs):
            result = None
            total_time = 0
            print(f"Running {func.__name__}() {self.repetitions} times")
            for _ in range(self.repetitions):
                start = time.perf_counter()
                result = func(*args, **kwargs)
                end = time.perf_counter()
                total_time += end - start
            average_time = total_time / self.repetitions
            print(
                f"{func.__name__}() takes "
                f"{average_time * 1000:.4f} ms on average"
            )
            return result

        return timer

In [96]:
@ExecutionTimer(10)
def square_numbers(numbers):
    return [number ** 2 for number in numbers]


print(square_numbers(list(range(100))))

Running square_numbers() 10 times
square_numbers() takes 0.0068 ms on average
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


### Implementing strategy pattern
The strategy design pattern allows you to define a family of similar algorithms and make them interchangeable at runtime. In other words, the pattern implements different solutions to a given type of problem, with each solution bundled in a specific object. Then, you can choose the appropriate solution dynamically.

Note: The strategy design pattern is also pretty useful in languages where functions aren’t first-class citizens. For example, in C++ or Java, using this pattern allows you to pass functions as arguments to other functions.

In [105]:
import json

import yaml

class JsonSerializer:
    def __call__(self, data):
        return json.dumps(data, indent=4)

class YamlSerializer:
    def __call__(self, data):
        return yaml.dump(data)

class DataSerializer:
    def __init__(self, serializing_strategy):
        self.serializing_strategy = serializing_strategy

    def serialize(self, data):
        return self.serializing_strategy(data)

In [103]:
data = {
    "name": "Jane Doe",
    "age": 30,
    "city": "Salt Lake City",
    "job": "Python Developer",
}

serializer = DataSerializer(JsonSerializer())
print(f"JSON:\n{serializer.serialize(data)}")

JSON:
{
    "name": "Jane Doe",
    "age": 30,
    "city": "Salt Lake City",
    "job": "Python Developer"
}


In [113]:
# Switch strategy
serializer.serializing_strategy = YamlSerializer()
print(f"YAML:\n{serializer.serialize(data)}")

YAML:
age: 30
city: Salt Lake City
job: Python Developer
name: Jane Doe

